In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.appName("Spark_CLOB_Split") \
.config("hive.metastore.sasl.enabled", "true") \
.enableHiveSupport() \
.getOrCreate()

In [ ]:
# 你看到的代码如果以 spark. 开头，你zhi需要用 sc. 开头就行了。
# 我这里已经给你 build 好了 sprak 的 context

In [3]:
# 定义 sparkContext,这个环境被包含在了 SparkSession 之中。
sc = spark.sparkContext

In [7]:
#1.创建 RDD，支持 tuple,dict,list类型
data = sc.parallelize([
    ('A','a'),
    {'B':10000},
    ['c','e','f']
]).collect()

In [11]:
#如果对数据集使用方法.collect()，将RDD的所有元素返回给驱动程序，可以访问对象中的数据
data[1]['B']

10000

In [ ]:
#　/home/yanglab/lianhaimiao/PySpark/datasets　
#　page_news.txt  VS14MORT.txt  VS14MORT.txt.gz

In [26]:
#读取外部文件
#sc.textFile(路径,n) n表示数据集被划分的分区个数，一般分为２～４个
#本地文件前加"file://"
data_from_file = sc.textFile('file:///home/yanglab/lianhaimiao/PySpark/datasets/VS14MORT.txt.gz',4)
#read data
data_from_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

In [27]:
#这段代码用来解析文件，使其可读
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]
    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs

In [28]:
data_from_file_conv = data_from_file.map(extractInformation)

In [35]:
#读取数据
data_from_file_conv.take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

1）map()转换

In [40]:
# 创建一个新的数据集，使死亡年份转换为数字
data_2014 = data_from_file_conv.map(lambda row:int(row[16]))
data_2014.take(10)   #take括号内的数字表示显示前几data_2014行

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

In [ ]:
#构造元组
data_2014_2 = data_from_file_conv.map(lambda row:(row[16],int(row[16])))
data_2014_2.take(5)

2）filter()转换：从数据集中选择满足特征标准的数据，形成新的数据

In [43]:
data_filtered = data_from_file_conv.filter(lambda row:row[16] == '2014' and row[21] == '0')
data_filtered.count()

22

3) flatMap()转换：和map()类似，但是返回的是一个扁平化结果，形成数组的形式

In [46]:
data_2014_flat =  data_from_file_conv.flatMap(lambda row:(row[16],int(row[16])+1))
data_2014_flat.take(5)

['2014', 2015, '2014', 2015, '2014']

4) distict() 返回指定列中，不同值的列表

In [48]:
distict_genter =  data_from_file_conv.map(lambda row:row[5]).distinct()
distict_genter.collect() #.collect() show result 

['M', 'F', '-99']

5) sample()转换：随机采样， 参数定义：sample(采样是否应该替换，返回数据的分数值，伪随机数产生的种子)

In [49]:
fraction = 0.1
data_sample = data_from_file_conv.sample(False,0.1,666)

In [50]:
print('origin data:{},sample data:{}'.format(data_from_file_conv.count(),data_sample.count()))

origin data:2631171,sample data:263247


6) .leftOuterJoin()转换：根据元组左侧都有的值，连接两个记录

In [51]:
rdd1 = sc.parallelize([('a',1),('b',2),('c',3)])
rdd2 = sc.parallelize([('a',1),('a',4),('b','5'),('d',6)])  
rdd3 = rdd1.leftOuterJoin(rdd2)   #这里要注意的是最后结果中没有d，因为是rdd1.letfOuterJoin(rdd2)
rdd3.collect()  #显示结果

[('b', (2, '5')), ('a', (1, 1)), ('a', (1, 4)), ('c', (3, None))]

In [52]:
#.join()
rdd4 = rdd1.join(rdd2)
rdd4.collect()   #注：这里只有a和b，没有存在none的c

[('b', (2, '5')), ('a', (1, 1)), ('a', (1, 4))]

In [53]:
#intersection()：返回两个RDD中相等的元素
rdd5 = rdd1.intersection(rdd2)
rdd5.collect()

[('a', 1)]

7) .reparttition():重新分区

In [54]:
rdd1 = rdd1.repartition(4)
len(rdd1.glom().collect())  #分区数目

4

In [58]:
#.take()
data_first = data_from_file_conv.take(1)
#.takeSample()
data_first_sample = data_from_file_conv.takeSample(False,1,667) #三个参数，第二个参数表示随便采样返回几个

8）reduce():使用指定的方法减少RDD中的元素

In [60]:
rdd1.map(lambda row:row[1]).reduce(lambda x,y:x+y)

6

In [62]:
#reduceByKey():在键-键基础上减少RDD的元素
data_key = sc.parallelize([('a',4),('a',6),('c',1),('d',3),('c',5),('b',7)],4)
data_key.reduceByKey(lambda x,y:x+y).collect()

[('c', 6), ('b', 7), ('a', 10), ('d', 3)]

9) count():统计RDD中的元素数量

In [63]:
data_key.count()

6

In [65]:
#countByKey():统计不同键的个数
data_key.countByKey().items() 
#.items() :返回一个包含 Dictionary 对象中所有条目的数组

dict_items([('a', 2), ('c', 2), ('d', 1), ('b', 1)])

10) .saveAsTextFile():使RDD保存为文本文件

In [66]:
data_key.saveAsTextFile('file:///home/yanglab/lianhaimiao/PySpark/datasets/data_key.txt')
#注：读取它需要解析文本文件

11) .foreach():按照一个接一个的方式，对每一条记录应用一个定义好的函数

In [67]:
def f(x):
    print(x)
data_key.foreach(f)  #打印到CLI

Spark中的转化是惰性的，它们只在操作被调用时应用。